<a target="_blank" href="https://colab.research.google.com/github/paulotguerra/QXD0178/blob/main/01.E0-Excercicio-Limpeza-de-dados.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## QXD0178 - Mineração de Dados
# Preparação da base de dados com Scikit-Learn

### Carga do conjunto de dados `food_coded.csv`

In [163]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

df = pd.read_csv("https://raw.githubusercontent.com/She-Codes-Now/Intro-to-Data-Science-with-R/master/food_coded.csv")

### RESOLUÇÃO DA ATIVIDADE

Irei verificar quais células possuem dados faltantes para que possam ser preenchidos.

In [164]:
missing_data = df.isnull().sum()
print(missing_data[missing_data > 0])

GPA                            2
calories_day                  19
calories_scone                 1
comfort_food                   1
comfort_food_reasons           2
comfort_food_reasons_coded    19
cook                           3
cuisine                       17
diet_current                   1
drink                          2
eating_changes                 3
employment                     9
exercise                      13
father_education               1
father_profession              3
fav_cuisine                    2
fav_food                       2
food_childhood                 1
healthy_meal                   1
ideal_diet                     1
income                         1
life_rewarding                 1
marital_status                 1
meals_dinner_friend            3
mother_education               3
mother_profession              2
on_off_campus                  1
persian_food                   1
self_perception_weight         1
soup                           1
sports    

Filtrando as colunas do tipo `object`:


In [165]:
object_columns = df.select_dtypes(include=['object']).columns
print(object_columns)

Index(['GPA', 'comfort_food', 'comfort_food_reasons', 'diet_current',
       'eating_changes', 'father_profession', 'fav_cuisine', 'food_childhood',
       'healthy_meal', 'ideal_diet', 'meals_dinner_friend',
       'mother_profession', 'type_sports', 'weight'],
      dtype='object')


Após fazer a filtragem, observei que as colunas `GPA` e `Weight` estão como tipo `object`, quando na verdade deveriam ser numéricas. Irei ajustá-las.

In [166]:
# Convertendo as colunas para float:

df['GPA'] = pd.to_numeric(df['GPA'], errors='coerce')
df['weight'] = pd.to_numeric(df['weight'], errors='coerce')

Filtrando as colunas númericas para que posteriormente elas sejam preenchidas utilizando `SimpleImputer`:

In [167]:
numeric_columns = df.select_dtypes(include=['number']).columns
print(numeric_columns)

Index(['GPA', 'Gender', 'breakfast', 'calories_chicken', 'calories_day',
       'calories_scone', 'coffee', 'comfort_food_reasons_coded', 'cook',
       'comfort_food_reasons_coded.1', 'cuisine', 'diet_current_coded',
       'drink', 'eating_changes_coded', 'eating_changes_coded1', 'eating_out',
       'employment', 'ethnic_food', 'exercise', 'father_education',
       'fav_cuisine_coded', 'fav_food', 'fries', 'fruit_day', 'grade_level',
       'greek_food', 'healthy_feeling', 'ideal_diet_coded', 'income',
       'indian_food', 'italian_food', 'life_rewarding', 'marital_status',
       'mother_education', 'nutritional_check', 'on_off_campus',
       'parents_cook', 'pay_meal_out', 'persian_food',
       'self_perception_weight', 'soup', 'sports', 'thai_food',
       'tortilla_calories', 'turkey_calories', 'veggies_day', 'vitamins',
       'waffle_calories', 'weight'],
      dtype='object')


In [168]:
# Aplicando SimpleImputer somente nas colunas numéricas
imputer = SimpleImputer(strategy="median")
df[numeric_columns] = imputer.fit_transform(df[numeric_columns])

In [169]:
# Tratando as colunas não númericas com SimpleImputer:
colunas_nao_num = [
    'diet_current', 'eating_changes', 'father_profession',
    'fav_cuisine', 'food_childhood', 'healthy_meal',
    'ideal_diet', 'meals_dinner_friend', 'mother_profession', 'type_sports'
]

imputer_cat = SimpleImputer(strategy='most_frequent')
df[colunas_nao_num] = imputer_cat.fit_transform(df[colunas_nao_num])

Os dados das colunas comfort_food_reasons e comfort_food estão inconsistentes porque os textos não estão separados nem escritos de maneira padronizada. Irei ajustar para cada palavra ser separada por vírgula e escritas em caracteres minúsculos, utilizando `Pipeline`.

In [170]:
# Função para padronizar texto
def padronizar_texto(text):
    if pd.isna(text):  # Verificar valores nulos
        return text
    text = text.replace("and", ",").replace("/", ",")
    text = text.replace(", ", ",").replace(" ,", ",")
    text = text.lower()
    return text

# Criando o pipeline
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='')),
])

# Aplicando o pipeline nas colunas
df['comfort_food_reasons'] = df['comfort_food_reasons'].fillna('').apply(padronizar_texto)
df['comfort_food'] = df['comfort_food'].fillna('').apply(padronizar_texto)


Verificando as informações do dataset após os passos anteriores, podemos observar que não há mais nenhum dado faltante no conjunto de dados:


In [171]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 61 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   GPA                           125 non-null    float64
 1   Gender                        125 non-null    float64
 2   breakfast                     125 non-null    float64
 3   calories_chicken              125 non-null    float64
 4   calories_day                  125 non-null    float64
 5   calories_scone                125 non-null    float64
 6   coffee                        125 non-null    float64
 7   comfort_food                  125 non-null    object 
 8   comfort_food_reasons          125 non-null    object 
 9   comfort_food_reasons_coded    125 non-null    float64
 10  cook                          125 non-null    float64
 11  comfort_food_reasons_coded.1  125 non-null    float64
 12  cuisine                       125 non-null    float64
 13  diet_